### Runtimes

This notebook is used to calculate the runtimes for the VI-based TVP-BVARS. It is based on an average of 16 runs (all the cores on the system) as the algorithm is deterministic and there is not much variation between the different simulated datasets. This is quite represenative for the comparison between the VAR, MCMC-based BVAR and MCMC-based TVP-BVAR.

In [2]:
%matplotlib inline
import time
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
from scipy import io
import pickle

# Own code
sys.path.append("../")
from utils.data_utils import generate_dgp_tvp_var, generate_matrices, generate_contemp_matrices
from utils.tvp_models import TVPVARModel, tvp_ar_contemp, tvp_ar_non_contemp

# Suppress scientific notation in numpy
np.set_printoptions(suppress=True)

In [2]:
def multiprocessing_run(prior, M):
    
    import time
    import os
    from multiprocessing import Pool, Array

    # They are going to be some disgusting warnings for the first iterations of the model (has to do with initialization)
    import warnings
    warnings.filterwarnings("ignore")

    np.random.seed(12345)
    n_iterations = 16

    iterations_set = np.arange(1,n_iterations+1,1)
    M = (M,)
    prior_list = [prior]#, "lasso_alternative", "horseshoe"]
    args_tuples = [x for sub_list in [x for sub_list in [[[(n, m, prior) for n in iterations_set] for m in M] for prior in prior_list] for x in sub_list] for x in sub_list]

    if __name__ == '__main__':

        pool = Pool()
        simulation_results = pool.starmap(simulation_run, args_tuples)

#### M = 3, $p_{0} = 0.40$ and $T = 200$

In [3]:
M = 3

In [ ]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    sparsity = 0.40
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [ ]:
simulation_run(1,3,"svss")

In [ ]:
%%time
multiprocessing_run("svss",M)

In [ ]:
%%time
multiprocessing_run("lasso_alternative",M)

In [ ]:
%%time
multiprocessing_run("horseshoe",M)

#### M = 3, $p_{0} = 0.20$ and $T = 200$

In [ ]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    sparsity = 0.20
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [ ]:
multiprocessing_run("svss",M)

In [ ]:
multiprocessing_run("lasso_alternative",M)

In [ ]:
multiprocessing_run("horseshoe",M)

#### M = 3, $p_{0} = 0.40$ and $T = 100$

In [4]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 100
    p = 1
    train = 75
    sparsity = 0.40
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [ ]:
multiprocessing_run("svss",M)

In [ ]:
multiprocessing_run("lasso_alternative",M)

In [5]:
multiprocessing_run("horseshoe",M)

Run: 4, M: 3 & prior: horseshoe -> MSD: 0.019439673226330164 | MSFE: 0.7469295631348887 | ALPL: -30.439087540648764 | elapsed time: 43.559984
Run: 1, M: 3 & prior: horseshoe -> MSD: 0.020774176336782045 | MSFE: 1.425240038764073 | ALPL: -31.470528771558122 | elapsed time: 44.611151
Run: 8, M: 3 & prior: horseshoe -> MSD: 0.025890535674318097 | MSFE: 3.1928023045485636 | ALPL: -29.30323282008829 | elapsed time: 45.156607
Run: 3, M: 3 & prior: horseshoe -> MSD: 0.03168751966098695 | MSFE: 1.748056904244041 | ALPL: -30.42702204570964 | elapsed time: 45.871619
Run: 7, M: 3 & prior: horseshoe -> MSD: 0.025884965678871434 | MSFE: 1.3650155935566282 | ALPL: -30.630914671944375 | elapsed time: 46.112856
Run: 16, M: 3 & prior: horseshoe -> MSD: 0.012997740555507972 | MSFE: 0.9596426873486433 | ALPL: -28.114881280795615 | elapsed time: 46.490316
Run: 15, M: 3 & prior: horseshoe -> MSD: 0.03439334062174744 | MSFE: 1.8681821958367313 | ALPL: -31.059439767560587 | elapsed time: 46.665417
Run: 13, M

#### M = 3, $p_{0} = 0.20$ and $T = 100$

In [6]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 100
    p = 1
    train = 75
    sparsity = 0.20
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [ ]:
multiprocessing_run("svss",M)

In [ ]:
multiprocessing_run("lasso_alternative",M)

In [7]:
multiprocessing_run("horseshoe",M)

Run: 1, M: 3 & prior: horseshoe -> MSD: 0.02218632769982002 | MSFE: 1.4325097658067159 | ALPL: -30.19845264487374 | elapsed time: 49.224456
Run: 16, M: 3 & prior: horseshoe -> MSD: 0.012305963949586022 | MSFE: 0.9983418845135883 | ALPL: -26.101298601276834 | elapsed time: 50.443254
Run: 7, M: 3 & prior: horseshoe -> MSD: 0.02572410057159032 | MSFE: 1.3443669920546282 | ALPL: -30.653016344881845 | elapsed time: 50.601245
Run: 8, M: 3 & prior: horseshoe -> MSD: 0.024680439212961525 | MSFE: 2.944083862765046 | ALPL: -30.558325724821668 | elapsed time: 50.995869
Run: 3, M: 3 & prior: horseshoe -> MSD: 0.03238797628539474 | MSFE: 1.8723611544937087 | ALPL: -29.174864931929612 | elapsed time: 51.100446
Run: 13, M: 3 & prior: horseshoe -> MSD: 0.022078697764642902 | MSFE: 1.5674800342783184 | ALPL: -27.920006501913477 | elapsed time: 51.753154
Run: 10, M: 3 & prior: horseshoe -> MSD: 0.046893737734210024 | MSFE: 1.1181871991815624 | ALPL: -24.78554945644995 | elapsed time: 52.159349
Run: 15, 

#### M = 7, $p_{0} = 0.40$ and $T = 200$

In [8]:
M = 7

In [9]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    sparsity = 0.40
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [ ]:
multiprocessing_run("svss",M)

In [ ]:
multiprocessing_run("lasso_alternative",M)

In [10]:
multiprocessing_run("horseshoe",M)

Run: 9, M: 7 & prior: horseshoe -> MSD: 0.016928410992058496 | MSFE: 5.225861650546065 | ALPL: -28.70809323625562 | elapsed time: 399.042783
Run: 3, M: 7 & prior: horseshoe -> MSD: 0.01182977638932928 | MSFE: 5.099454789383466 | ALPL: -29.57658811887578 | elapsed time: 401.890295
Run: 5, M: 7 & prior: horseshoe -> MSD: 0.014956496147772561 | MSFE: 4.829312918703357 | ALPL: -29.96162086227478 | elapsed time: 411.085203
Run: 12, M: 7 & prior: horseshoe -> MSD: 0.017997511792161223 | MSFE: 4.981670390884561 | ALPL: -26.267697604382523 | elapsed time: 412.029916
Run: 6, M: 7 & prior: horseshoe -> MSD: 0.015966505298220492 | MSFE: 4.567937785484504 | ALPL: -30.690444478155563 | elapsed time: 419.843017
Run: 15, M: 7 & prior: horseshoe -> MSD: 0.013516900335510963 | MSFE: 7.589761242218486 | ALPL: -31.42825615806749 | elapsed time: 422.927362
Run: 4, M: 7 & prior: horseshoe -> MSD: 0.014391339045195213 | MSFE: 8.998548618860866 | ALPL: -31.789138079969344 | elapsed time: 424.279833
Run: 11, 

#### M = 7, $p_{0} = 0.20$ and $T = 200$

In [11]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 200
    p = 1
    train = 175
    sparsity = 0.20
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [ ]:
multiprocessing_run("svss",M)

In [ ]:
multiprocessing_run("lasso_alternative",M)

In [12]:
multiprocessing_run("horseshoe",M)

Run: 6, M: 7 & prior: horseshoe -> MSD: 0.01612992878328063 | MSFE: 5.330934337967733 | ALPL: -31.148307546059826 | elapsed time: 417.463613
Run: 9, M: 7 & prior: horseshoe -> MSD: 0.015940774648277825 | MSFE: 4.3359153779298385 | ALPL: -29.542868878297426 | elapsed time: 431.710003
Run: 3, M: 7 & prior: horseshoe -> MSD: 0.012181885291474936 | MSFE: 5.1085860659115205 | ALPL: -30.020010166421873 | elapsed time: 433.591378
Run: 12, M: 7 & prior: horseshoe -> MSD: 0.019852174083289143 | MSFE: 2.703373689912941 | ALPL: -30.527444724928973 | elapsed time: 434.962302
Run: 15, M: 7 & prior: horseshoe -> MSD: 0.016477011479126835 | MSFE: 7.646747992730611 | ALPL: -31.80101197596168 | elapsed time: 439.630941
Run: 7, M: 7 & prior: horseshoe -> MSD: 0.01449422923193236 | MSFE: 2.265541616635108 | ALPL: -29.761699252515285 | elapsed time: 440.036047
Run: 1, M: 7 & prior: horseshoe -> MSD: 0.014031258854691349 | MSFE: 3.468412708854636 | ALPL: -31.14347422793547 | elapsed time: 441.828479
Run: 4

#### M = 7, $p_{0} = 0.40$ and $T = 100$

In [13]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 100
    p = 1
    train = 75
    sparsity = 0.40
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [ ]:
multiprocessing_run("svss",M)

In [ ]:
multiprocessing_run("lasso_alternative",M)

In [14]:
multiprocessing_run("horseshoe",M)

Run: 16, M: 7 & prior: horseshoe -> MSD: 0.013215358772422988 | MSFE: 2.810707171517505 | ALPL: -30.53863186690779 | elapsed time: 163.860071
Run: 11, M: 7 & prior: horseshoe -> MSD: 0.024040227632483882 | MSFE: 1.945646174570384 | ALPL: -30.55978470432386 | elapsed time: 170.885175
Run: 9, M: 7 & prior: horseshoe -> MSD: 0.03014011810587774 | MSFE: 2.6701107515310474 | ALPL: -29.294834771013647 | elapsed time: 172.520821
Run: 14, M: 7 & prior: horseshoe -> MSD: 0.02043264998496976 | MSFE: 2.046214635241695 | ALPL: -30.917344632496334 | elapsed time: 172.577168
Run: 7, M: 7 & prior: horseshoe -> MSD: 0.020692940861308715 | MSFE: 2.052773014684959 | ALPL: -29.319726765264733 | elapsed time: 173.836554
Run: 8, M: 7 & prior: horseshoe -> MSD: 0.021590348611518814 | MSFE: 2.746269645691959 | ALPL: -29.332905862466458 | elapsed time: 175.052384
Run: 5, M: 7 & prior: horseshoe -> MSD: 0.01709638020788528 | MSFE: 1.709835406275511 | ALPL: -29.556340303028456 | elapsed time: 175.667401
Run: 2,

#### M = 7, $p_{0} = 0.20$ and $T = 100$

In [15]:
def simulation_run(run, M, prior):
    
    start_time = time.time()
    
    # Fixed for simulation
    T = 100
    p = 1
    train = 75
    sparsity = 0.20
    
    y = np.loadtxt(f'../simulations/datasets/y_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    coefficients = np.loadtxt(f'../simulations/datasets/coefficients_{M}_{T}_{p}_{sparsity}_{run}_het_py.csv', delimiter=",")
    
    y_matrix_contemp, X_matrix_contemp = generate_contemp_matrices(T, M, p, y)
    
    # Contemperanous values added 
    msfe_contemp, alpl_contemp, mt1t_full_contemp, mt1t_coeff_contemp, sigma_contemp, ar_model = tvp_ar_contemp(T, M, p, train, X_matrix_contemp, y_matrix_contemp, prior, print_status=False, iterations=100)
    msd_contemp = np.mean((mt1t_coeff_contemp - coefficients[:,1:train])**2)
    
    elapsed_time = time.time() - start_time
    
    print(f'Run: {run}, M: {M} & prior: {prior} -> MSD: {msd_contemp} | MSFE: {msfe_contemp.mean()} | ALPL: {alpl_contemp.mean()} | elapsed time: {round(elapsed_time,6)}')
    
    return [msfe_contemp, alpl_contemp, msd_contemp, M, prior]

In [ ]:
multiprocessing_run("svss",M)

In [ ]:
multiprocessing_run("lasso_alternative",M)

In [16]:
multiprocessing_run("horseshoe",M)

Run: 16, M: 7 & prior: horseshoe -> MSD: 0.014940289504365522 | MSFE: 2.857558442570216 | ALPL: -31.312002717861684 | elapsed time: 171.068122
Run: 14, M: 7 & prior: horseshoe -> MSD: 0.01912192360081559 | MSFE: 2.1677704373467805 | ALPL: -31.40706201370316 | elapsed time: 171.548735
Run: 4, M: 7 & prior: horseshoe -> MSD: 0.01857582602302756 | MSFE: 2.2016560070237325 | ALPL: -30.938936514908658 | elapsed time: 172.039924
Run: 5, M: 7 & prior: horseshoe -> MSD: 0.019315060020416684 | MSFE: 1.536930630424011 | ALPL: -29.768064030632758 | elapsed time: 173.624417
Run: 8, M: 7 & prior: horseshoe -> MSD: 0.022912516286149515 | MSFE: 2.394564303461246 | ALPL: -29.2181806976473 | elapsed time: 173.909481
Run: 11, M: 7 & prior: horseshoe -> MSD: 0.025032487717507868 | MSFE: 1.5880921379086974 | ALPL: -30.558949745456758 | elapsed time: 175.586936
Run: 9, M: 7 & prior: horseshoe -> MSD: 0.027826357449088786 | MSFE: 2.6596930179341296 | ALPL: -29.31921628253898 | elapsed time: 175.799907
Run: 